In [52]:
!pip install yfinance
!pip install torchsummaryX
!pip install wandb -q

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [53]:
%matplotlib inline

import numpy as np
from matplotlib import pyplot as plt
import time
import os
import yfinance as yf
import pandas as pd
import datetime
import torch
import torch.nn as nn
from torchsummaryX import summary
from torch.autograd import Variable
from sklearn.preprocessing import MinMaxScaler
from torch.utils.data import Dataset, DataLoader
from tqdm import tqdm
import gc
from sklearn.model_selection import KFold
from torch.utils.data import Dataset, DataLoader,TensorDataset,random_split,SubsetRandomSampler, ConcatDataset

device = 'cuda' if torch.cuda.is_available() else 'cpu'

In [54]:
import os
from google.colab import drive
drive.mount('/content/drive')
os.getcwd()

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


'/content'

In [55]:
import wandb
wandb.login(key="03916b709446813b51f72a1b29a2854a9dd9e3f7")

wandb: WARNING Calling wandb.login() after wandb.init() has no effect.


True

In [56]:
# Training Config
config = {
    "batch_size" : 32,
    "lr" : 1e-3,
    "epochs" : 100,
    "look_back" : 60,
    "feature_dim" : 6,
    "hidden_dim" : 64,
    "output_dim" : 1,
    "num_layers" : 3,
    "dropout" : 0.5
}

In [57]:
pd.set_option('display.max_rows', 1000)
pd.set_option('display.max_columns', 100)
pd.set_option('display.width', 200)

class VolatiltiyData:

    def __init__(self, tickers, start_year, start_month, start_date, end_year,
                 end_month, end_date, freq, scaling_factor, windows, y_window):
        self.tickers = tickers
        self.start = datetime.datetime(start_year, start_month, start_date)
        self.end = datetime.datetime(end_year, end_month, end_date)
        self.freq = freq
        self.scaling_factor = scaling_factor
        # 过去几天的vol
        self.windows = windows
        # Predict几天的vol
        self.y_window = y_window 

    def get_data(self):
        """
        Output:
        raw data -> ['Open', 'High', 'Low', 'Close', 'Adj Close', 'Volume', 
                     'Log Adj Close', 'Log Return', 'Log Volume', 'Log Volume Chg', 'Log Range', '10-day-vol']
        """
        data = yf.download(self.tickers, start = self.start, end = self.end, interval = self.freq)
        data["Log Adj Close"] = np.log(data["Adj Close"])
        data["Log Return"] = np.insert(np.diff(data["Log Adj Close"]), 0, 0) * self.scaling_factor
        data["Log Volume"] = np.log(data["Volume"])
        data["Log Volume Chg"] = np.insert(np.diff(data["Log Volume"]), 0, 0)
        data["Log Range"] = np.log(data["High"] / data["Low"]) * self.scaling_factor
        data["10-day-vol"] = data["Log Return"].rolling(10).std(ddof=0)
        data["30-day-vol"] = data["Log Return"].rolling(30).std(ddof=0)

        return data

    def get_vix_data(self):
        """
        Output:
        raw vix data -> ['Open', 'High', 'Low', 'Close', 'Adj Close', 'Volume']
        """
        data = yf.download("^VIX", start = self.start, end = self.end, interval = self.freq)
        
        return data

    def prepare_data(self, *args):
        """
        Prepare data for training. Select features that are needed. Perform necessary normalization.
        
        For volatiltiy data, we need to remove initial n days since we need at least n days to calculate volatiltiy.

        Input:
        Multiple types of data:
        Basic data: 10-day-vol, Log Volume Chg, Log Range
        Optional data: VIX?

        Output:
        dataset ->  (samples, features)
        scalar  ->  scalar for our normalization

        Current features:
        [10-day-vol, Log Return, Log Volume Chg, Log Range, 30-day-vol, VIX]
        Update based on correlation [10-day-vol, Log Range, 30-day-vol, VIX]
        """
        data = args[0]
        vol_10 = data['10-day-vol'].values.reshape(-1, 1)
        log_return = data['Log Return'].values.reshape(-1, 1)
        log_vlmchg = data['Log Volume Chg'].values.reshape(-1, 1)
        log_range = data['Log Range'].values.reshape(-1, 1)
        vol_30 = data['30-day-vol'].values.reshape(-1, 1)
        
        dataset = vol_10
        dataset = np.append(dataset, log_return, axis=1)
        dataset = np.append(dataset, log_vlmchg, axis = 1)
        dataset = np.append(dataset, log_range, axis = 1)
        dataset = np.append(dataset, vol_30, axis = 1)
        
        # For appending more types of data -> VIX
        for i in range(1, len(args)):
            extra_data = args[i]['Adj Close'].values.reshape(-1, 1)
            dataset = np.append(dataset, extra_data, axis = 1)

        dataset = dataset[31:]
        
        # normalize the dataset
        scaler = MinMaxScaler(feature_range=(0, 1))
        dataset = scaler.fit_transform(dataset)
        return dataset, vol_10, scaler

volatiltiy_data = VolatiltiyData(
                      "^GSPC",
                      1990, 1, 2,
                      2022, 12, 12,
                      freq="1d",
                      scaling_factor=100,
                      windows=[10], # 过去几天的vol
                      y_window=10 # Predict几天的vol
                  )

data = volatiltiy_data.get_data()
# print(np.shape(data))
# print(data)
# print("#######################################################################")
vix_data = volatiltiy_data.get_vix_data()
# print(np.shape(vix_data))
# print(vix_data)
# print("#######################################################################")
dataset, vol_10, scaler = volatiltiy_data.prepare_data(data, vix_data)
print(np.shape(dataset))
# print(dataset)

[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
(8270, 6)


In [58]:
df = data.copy()

df['VIX'] = vix_data['Adj Close']
df['Y'] = list(df['10-day-vol'])[1:] + [0]
df.iloc[31:, :].corr()

,Open,High,Low,Close,Adj Close,Volume,Log Adj Close,Log Return,Log Volume,Log Volume Chg,Log Range,10-day-vol,30-day-vol,VIX,Y
Open,1.000000,0.999939,0.999908,0.999841,0.999841,0.606128,0.928038,-0.007309,0.662764,-0.000161,-0.030450,0.025482,0.039784,0.005281,0.025495
High,0.999939,1.000000,0.999878,0.999916,0.999916,0.607602,0.928124,-0.000608,0.663549,0.000218,-0.025355,0.030240,0.043871,0.008986,0.030176
Low,0.999908,0.999878,1.000000,0.999924,0.999924,0.603854,0.928091,0.001205,0.661675,-0.001773,-0.038569,0.020281,0.035356,-0.000993,0.020006
Close,0.999841,0.999916,0.999924,1.000000,1.000000,0.605676,0.928174,0.007725,0.662611,-0.001164,-0.032200,0.025501,0.039853,0.003284,0.025198
Adj Close,0.999841,0.999916,0.999924,1.000000,1.000000,0.605676,0.928174,0.007725,0.662611,-0.001164,-0.032200,0.025501,0.039853,0.003284,0.025198
Volume,0.606128,0.607602,0.603854,0.605676,0.605676,1.000000,0.676620,-0.025952,0.909445,0.107652,0.287947,0.334698,0.339552,0.312287,0.339322
Log Adj Close,0.928038,0.928124,0.928091,0.928174,0.928174,0.676620,1.000000,0.006101,0.823790,-0.001052,0.007233,0.063723,0.080503,0.022871,0.063375
Log Return,-0.007309,-0.000608,0.001205,0.007725,0.007725,-0.025952,0.006101,1.000000,-0.012735,-0.028871,-0.104016,-0.005016,-0.001999,-0.134462,-0.028897
Log Volume,0.662764,0.663549,0.661675,0.662611,0.662611,0.909445,0.823790,-0.012735,1.000000,0.084669,0.196749,0.240805,0.255808,0.200707,0.242190
Log Volume Chg,-0.000161,0.000218,-0.001773,-0.001164,-0.001164,0.107652,-0.001052,-0.028871,0.084669,1.000000,0.160535,-0.009776,-0.008562,0.009929,-0.009615


In [59]:
class DataLoaderForVolatilityModeling(DataLoader):
    """
    Construct dataloader
  
    The label for our data is the next 10-day volatility after our input.

    Output:
    input -> (batch_size, seq_len, feature_size)
    label -> (batch_size, 1, feature_size) 
    """
    def __init__(self, dataset, vol_10, batch_size, sequence_length, shuffle=True, feature_size=1):
        self.dataset = dataset
        self.vol_10 = vol_10
        self.batch_size = batch_size
        self.shuffle = shuffle
        self.sequence_length = sequence_length
        self.num_batches = (np.shape(self.dataset)[0]-10) // self.batch_size
        self.feature_size = feature_size

    def __len__(self):
        return self.num_batches - self.sequence_length

    def __iter__(self):
        # group the sequence into batches
        x = torch.from_numpy(np.reshape(self.dataset[:self.num_batches*self.batch_size, :], (self.batch_size, self.num_batches, config['feature_dim']))).type(torch.float32)
        y = torch.from_numpy(np.reshape(self.vol_10[10:self.num_batches*self.batch_size+10, :], (self.batch_size, -1))).type(torch.float32)

        # return a tuple of (input, label) on every iteration with yield
        index = 0
        while index+self.sequence_length < self.num_batches:
            time_steps = self.sequence_length
            input = x[:, index:index+time_steps, :]
            label = y[:, index+time_steps].view(self.batch_size, 1)
            index += 1
            yield input, label

In [60]:
# sanity check
dataloader = DataLoaderForVolatilityModeling(dataset, vol_10, batch_size=config["batch_size"], sequence_length=config["look_back"], feature_size=config["feature_dim"])
tmp = iter(dataloader)
input, label = next(tmp) 
print(f'input shape = {np.shape(input)}')
print(f'label shape = {np.shape(label)}')

input shape = torch.Size([32, 60, 6])
label shape = torch.Size([32, 1])


In [61]:
# model
class Model(nn.Module):
    """
    Construct model architecture
    """
    def __init__(self, feature_dim, hidden_dim, output_dim):
        super(Model, self).__init__()
        self.lstm = torch.nn.GRU(input_size=feature_dim, hidden_size=hidden_dim, num_layers=config["num_layers"], 
                                  dropout=config["dropout"], batch_first=True, bias=True, bidirectional=True)
        self.relu = torch.nn.ReLU()
        self.linear = torch.nn.Linear(in_features=hidden_dim*2, out_features=output_dim, bias=True)

    def forward(self, x):
        x, h = self.lstm(x)
        x = x[:, -1, :]
        x = self.relu(x)
        x = self.linear(x)
        return x

In [62]:
# sanity check
torch.cuda.empty_cache()

model = Model(config['feature_dim'], config['hidden_dim'], config['output_dim']).to(device)
prediction = model(input.to(device))
print(f'shape of prediction : {prediction.shape}')
print(f'shape of label : {label.shape}')

summary(model, input.to(device))

shape of prediction : torch.Size([32, 1])
shape of label : torch.Size([32, 1])
         Kernel Shape   Output Shape   Params Mult-Adds
Layer                                                  
0_lstm              -  [32, 60, 128]  176.64k  174.336k
1_relu              -      [32, 128]        -         -
2_linear     [128, 1]        [32, 1]    129.0     128.0
---------------------------------------------------------
                        Totals
Total params          176.769k
Trainable params      176.769k
Non-trainable params       0.0
Mult-Adds             174.464k


/usr/local/lib/python3.8/dist-packages/torchsummaryX/torchsummaryX.py:101: FutureWarning: Dropping of nuisance columns in DataFrame reductions (with 'numeric_only=None') is deprecated; in a future version this will raise TypeError.  Select only valid columns before calling the reduction.
  df_sum = df.sum()


,Kernel Shape,Output Shape,Params,Mult-Adds
Layer,,,,
0_lstm,-,"[32, 60, 128]",176640.0,174336.0
1_relu,-,"[32, 128]",NaN,NaN
2_linear,"[128, 1]","[32, 1]",129.0,128.0


In [63]:
# Prepare loader
train_size = int(len(dataset) * 0.5)
val_size = int(len(dataset) * 0.2)
test_size = len(dataset) - train_size - val_size
train_data, val_data, test_data = dataset[0:train_size,:], dataset[train_size:train_size+val_size,:], dataset[train_size+val_size:len(dataset),:]
train_vol_10, val_vol_10, test_vol_10 = vol_10[0:train_size,:], vol_10[train_size:train_size+val_size,:], vol_10[train_size+val_size:len(dataset),:]

train_loader = DataLoaderForVolatilityModeling(train_data, train_vol_10, batch_size=config['batch_size'], sequence_length=config['look_back'], shuffle=True)
val_loader = DataLoaderForVolatilityModeling(val_data, val_vol_10, batch_size=10, sequence_length=config['look_back'], shuffle=False)
test_loader = DataLoaderForVolatilityModeling(test_data, test_vol_10, batch_size=1, sequence_length=config['look_back'], shuffle=False)

# Prepare optimizer, criterion, and scheduler_lr
optimizer = torch.optim.AdamW(model.parameters(), lr=config['lr'], weight_decay = 5e-5) # What goes in here?
criterion = torch.nn.MSELoss()
scheduler_lr = torch.optim.lr_scheduler.ReduceLROnPlateau(optimizer, mode="min", factor=0.75, patience=2, verbose=True)

In [64]:
# Evaluate
def evaluate(val_loader, model, criterion):
    model.eval()

    batch_bar = tqdm(total=len(val_loader), dynamic_ncols=True, position=0, leave=False, desc="Val")

    val_loss = 0

    for i, (input, target) in enumerate(val_loader):

        input, target = input.to(device), target.to(device)

        with torch.inference_mode():
            prediction = model(input)

        loss = criterion(prediction.flatten(), target.flatten())

        val_loss += loss.item()
        # print(loss.item())

        batch_bar.set_postfix(
            loss = f"{val_loss/ (i+1):.4f}",
            lr = f"{curr_lr}"
        )

        batch_bar.update()

        torch.cuda.empty_cache()
        del input
        del target

    batch_bar.close()
    val_loss /= len(dataloader)

    return val_loss

In [65]:
# Train Step
def train_step(train_loader, model, optimizer, criterion):
    batch_bar = tqdm(total=len(train_loader), dynamic_ncols=True, leave=False, position=0, desc='Train')
    train_loss = 0
    model.train()

    for i, (input, target) in enumerate(train_loader):
        input, target = input.to(device), target.to(device)
        curr_lr = optimizer.param_groups[0]['lr']

        optimizer.zero_grad()
        prediction = model(input)
        # print(f'prediction : {prediction}')
        # print(f'target : {target}')
        loss = criterion(prediction.flatten(), target.flatten())
        # print(f'loss : {loss.item()}')

        loss.backward()
        optimizer.step()

        train_loss += loss.item()

        batch_bar.set_postfix(
            loss = f"{train_loss/ (i+1):.4f}",
            lr = f"{curr_lr}"
        )

        batch_bar.update()

        torch.cuda.empty_cache()
        del input
        del target
    
    batch_bar.close()
    train_loss /= len(train_loader)
    
    return train_loss

In [67]:
Y = vol_10[:-31, :]

n_splits=10

kf = KFold(n_splits)
kf.get_n_splits(Y)

losses = []

for i, (train_index, val_index) in enumerate(kf.split(Y)):
    train_X = np.take(dataset, train_index, axis=0)
    train_Y = np.reshape(np.take(Y, train_index), (-1, 1))
    val_X = np.take(dataset, val_index, axis=0)
    val_Y = np.reshape(np.take(Y, val_index), (-1, 1))
    train_loader = DataLoaderForVolatilityModeling(train_X, train_Y, batch_size=config['batch_size'], sequence_length=config['look_back'], shuffle=True)
    val_loader = DataLoaderForVolatilityModeling(val_X, val_Y, batch_size=10, sequence_length=config['look_back'], shuffle=False)
    
    model = Model(config['feature_dim'], config['hidden_dim'], config['output_dim']).to(device)
    optimizer = torch.optim.AdamW(model.parameters(), lr=config['lr'], weight_decay = 5e-5) # What goes in here?
    criterion = torch.nn.MSELoss()
    scheduler_lr = torch.optim.lr_scheduler.ReduceLROnPlateau(optimizer, mode="min", factor=0.75, patience=2, verbose=True)

    torch.cuda.empty_cache()
    gc.collect()

    best_val_loss = 100
    train_loss, val_loss = 0, 0

    # Initialize wandb
    run = wandb.init(
        name = "GRU_mult_feats_fold{:d}/{:d}".format(i+1, n_splits), ## Wandb creates random run names if you skip this field
        reinit = True, ### Allows reinitalizing runs when you re-run this cell
        # run_id = ### Insert specific run id here if you want to resume a previous run
        # resume = "must" ### You need this to resume previous runs, but comment out reinit = True when using this
        project = "10701_Volatility_Prediction", ### Project should be created in your wandb account 
        config = config ### Wandb Config for your run
    )

    for epoch in range(config["epochs"]):

        
        curr_lr = optimizer.param_groups[0]['lr']

        train_loss = train_step(train_loader, model, optimizer, criterion)

        val_loss = evaluate(val_loader, model, criterion)

        scheduler_lr.step(train_loss)

        print("\nEpoch {}/{}: \n\t Train Loss {:.07f}\t Eval Loss {:.07f}\t Learning Rate {:.04f}\t".format(
              epoch + 1,
              config['epochs'],
              train_loss,
              val_loss,
              curr_lr))
        
        wandb.log({"train_loss":train_loss, 'val_loss': val_loss, "learning_Rate": curr_lr})

        if val_loss < best_val_loss:
            best_val_loss = val_loss

    losses.append(best_val_loss)

    run.finish()

Exception ignored in: <function WeakSet.__init__.<locals>._remove at 0x7f35fe825670>
Traceback (most recent call last):
  File "/usr/lib/python3.8/_weakrefset.py", line 38, in _remove
    def _remove(item, selfref=ref(self)):
KeyboardInterrupt: 



Epoch 1/100: 
	 Train Loss 0.2279211	 Eval Loss 0.0073330	 Learning Rate 0.0010	



Epoch 2/100: 
	 Train Loss 0.0759581	 Eval Loss 0.0053578	 Learning Rate 0.0010	



Epoch 3/100: 
	 Train Loss 0.0592404	 Eval Loss 0.0035148	 Learning Rate 0.0010	



Epoch 4/100: 
	 Train Loss 0.0543657	 Eval Loss 0.0027407	 Learning Rate 0.0010	


...


Epoch 8/100: 
	 Train Loss 0.0225460	 Eval Loss 0.0009564	 Learning Rate 0.0010	



Epoch 9/100: 
	 Train Loss 0.0220455	 Eval Loss 0.0010066	 Learning Rate 0.0010	



Epoch 10/100: 
	 Train Loss 0.0201826	 Eval Loss 0.0007477	 Learning Rate 0.0010	


Train:  62%|██████▏   | 107/172 [00:01<00:01, 64.58it/s, loss=0.0150, lr=0.001]

KeyboardInterrupt: ignored

In [ ]:
print(losses)

In [ ]:
history = {'train_loss': [], 'test_loss': [],'train_acc':[],'test_acc':[]}

for fold, (train_idx,val_idx) in enumerate(splits.split(np.arange(len(dataset)))):

    print('Fold {}'.format(fold + 1))

    train_sampler = SubsetRandomSampler(train_idx)
    test_sampler = SubsetRandomSampler(val_idx)
    train_loader = DataLoader(dataset, batch_size=batch_size, sampler=train_sampler)
    test_loader = DataLoader(dataset, batch_size=batch_size, sampler=test_sampler)
    
    model = Model(config['feature_dim'], config['hidden_dim'], config['output_dim']).to(device)
    optimizer = optim.Adam(model.parameters(), lr=0.002)

    for epoch in range(num_epochs):
        train_loss, train_correct=train_epoch(model,device,train_loader,criterion,optimizer)
        test_loss, test_correct=valid_epoch(model,device,test_loader,criterion)

        train_loss = train_loss / len(train_loader.sampler)
        train_acc = train_correct / len(train_loader.sampler) * 100
        test_loss = test_loss / len(test_loader.sampler)
        test_acc = test_correct / len(test_loader.sampler) * 100

        print("Epoch:{}/{} AVG Training Loss:{:.3f} AVG Test Loss:{:.3f} AVG Training Acc {:.2f} % AVG Test Acc {:.2f} %".format(epoch + 1,
                                                                                                             num_epochs,
                                                                                                             train_loss,
                                                                                                             test_loss,
                                                                                                             train_acc,
                                                                                                             test_acc))
        history['train_loss'].append(train_loss)
        history['test_loss'].append(test_loss)
        history['train_acc'].append(train_acc)
        history['test_acc'].append(test_acc)   


In [ ]:
best_val_loss = 100

# Train loop
torch.cuda.empty_cache()
gc.collect()

train_loss, val_loss = 0, 0

for epoch in range(config["epochs"]):
    
    curr_lr = optimizer.param_groups[0]['lr']

    train_loss = train_step(train_loader, model, optimizer, criterion)

    val_loss = evaluate(val_loader, model, criterion)

    scheduler_lr.step(train_loss)

    print("\nEpoch {}/{}: \n\t Train Loss {:.07f}\t Eval Loss {:.07f}\t Learning Rate {:.04f}\t".format(
          epoch + 1,
          config['epochs'],
          train_loss,
          val_loss,
          curr_lr))
    
    wandb.log({"train_loss":train_loss, 'val_loss': val_loss, "learning_Rate": curr_lr})

    if val_loss < best_val_loss:
        path = '/content/drive/MyDrive/10701/checkpoint/checkpoint_eval_loss_haha'.format(val_loss)
        torch.save({'model_state_dict': model.state_dict()}, path)
        best_val_loss = val_loss

wandb.save(path)
run.finish()

In [ ]:
checkpoint = torch.load(path)
model.load_state_dict(checkpoint['model_state_dict'])

In [ ]:
# Prediction
def predict(test_loader, model):
    model.eval()
    preds, targets = [], []
    test_loss = 0

    for i, (input, target) in enumerate(test_loader):

        input, target = input.to(device), target.to(device)

        with torch.inference_mode():
            prediction = model(input)

        test_loss += criterion(prediction.flatten(), target.flatten())

        # Note: Each loaded data is a column
        prediction = prediction.cpu().detach().numpy() # (batch_size, feature_size)
        target = target.cpu().detach().numpy() # (batch_size, feature_size)

        preds.append(prediction.flatten()[0])
        targets.append(target.flatten()[0])

        del input
        del target
    
    test_loss /= len(test_loader)

    return preds, targets, test_loss

In [ ]:
torch.cuda.empty_cache()
gc.collect()

preds, targets, test_loss = predict(test_loader, model)

loss = 'Eval loss : {:.07}'.format(best_val_loss)
print(loss)

loss = 'Test loss : {:.07}'.format(test_loss)
print(loss)

In [ ]:
# Mean absolute percentage error
MAPE = 0
for i in range(len(preds)):
    MAPE += np.abs((targets[i]-preds[i])/targets[i])

print(MAPE/len(preds))

In [ ]:
df = data.copy()
df = df.tail(len(preds))
df.drop(df.columns, inplace=True, axis=1)
df['preds'] = preds

# Generate output
plt.figure(figsize=(13,9))
plt.plot(df.index, preds, label="Prediction", color="red")
plt.plot(df.index, targets, label="Targets", color="blue")
plt.xlabel("Years")
plt.ylabel("10-Day Volatility")
plt.legend(loc="upper left")
plt.show()

In [ ]:
# Prediction for the entire dataset
data_loader = DataLoaderForVolatilityModeling(dataset, vol_10,batch_size=1, sequence_length=config['look_back'], shuffle=False)

# Make predictions
preds, targets, test_loss = predict(data_loader, model)

# Generate output
plt.figure(figsize=(13,9))
plt.plot(data.index[:len(preds)], preds, label="Prediction", color="red")
plt.plot(data.index[:len(targets)], targets, label="Targets", color="blue")
plt.xlabel("Years")
plt.ylabel("10-Day Volatility")
plt.legend(loc="upper left")
plt.show()

loss = 'Test loss : {:.07}'.format(test_loss)
print(loss)